In [1]:
!ls llm_datasets

Chinese-LLaVA-Vision-Instructions  LLaVA-CC3M-Pretrain-595K  LLaVA-Pretrain


In [16]:
!ls Chinese-LLaVA-Vision-Instructions/LLaVA-CC3M-Pretrain-595K/chat-translated.json

# llm_datasets/Chinese-LLaVA-Vision-Instructions/LLaVA-Instruct-150K/translated/llava_instruct_150k.json  
# llm_datasets/Chinese-LLaVA-Vision-Instructions/LLaVA-Instruct-150K/translated/llava_instruct_80k.json

Chinese-LLaVA-Vision-Instructions/LLaVA-CC3M-Pretrain-595K/chat-translated.json


In [20]:
!head -n 50  Chinese-LLaVA-Vision-Instructions/LLaVA-CC3M-Pretrain-595K/chat-translated.json

[
  {
    "id": "GCC_train_002582585",
    "image": "GCC_train_002582585.jpg",
    "conversations": [
      {
        "from": "human",
        "value": "提供给定图像的简要描述。\n<image>"
      },
      {
        "from": "gpt",
        "value": "橄榄油是自由使用的健康成分。"
      }
    ]
  },
  {
    "id": "GCC_train_002429825",
    "image": "GCC_train_002429825.jpg",
    "conversations": [
      {
        "from": "human",
        "value": "<image>\n写一篇简短但有益的图片摘要."
      },
      {
        "from": "gpt",
        "value": "随机旋转的薄金符号的3D向量豪华字母。"
      }
    ]
  },
  {
    "id": "GCC_train_000190697",
    "image": "GCC_train_000190697.jpg",
    "conversations": [
      {
        "from": "human",
        "value": "分享提供图像的简要解释。\n<image>"
      },
      {
        "from": "gpt",
        "value": "气候变化——岩石的物理和化学分解。"
      }
    ]
  },
  {
    "id": "GCC_train_002503829",
    "image": "GCC_train_002503829.jpg",
    "conversations": [
      {
        "from": "human",
        "value": "简要, 清晰地说明所显示的图片.\n<image>"


In [10]:
!pip install pickleshare


[notice] A new release of pip is available: 23.3.1 -> 23.3.2
[notice] To update, run: python -m pip install --upgrade pip


In [11]:
%cd llm_datasets

[Errno 2] No such file or directory: 'llm_datasets'
/workspace/llm_datasets


/usr/local/lib/python3.10/dist-packages/IPython/core/magics/osm.py:393: UserWarning: using bookmarks requires you to install the `pickleshare` library.
  bkms = self.shell.db.get('bookmarks', {})


In [ ]:
# merge chinese to llava-pretrain
# merge 
# /workspace/llm_datasets/Chinese-LLaVA-Vision-Instructions/LLaVA-CC3M-Pretrain-595K/chat-translated.json
# to
# /workspace/llm_datasets/LLaVA-Pretrain/blip_laion_cc_sbu_558k.json

In [18]:
# backup
!cp /workspace/llm_datasets/LLaVA-Pretrain/blip_laion_cc_sbu_558k.json /workspace/llm_datasets/LLaVA-Pretrain/blip_laion_cc_sbu_558k.json.backup

In [7]:
from pathlib import Path
import json
from tqdm import tqdm

sources = [
    {
        "json": Path('/workspace/llm_datasets/Chinese-LLaVA-Vision-Instructions/LLaVA-CC3M-Pretrain-595K/chat-translated.json'),
        "image": Path('/workspace/llm_datasets/LLaVA-Pretrain'),
        "flag": "cc3m",
    },
    {
        "json": Path('/workspace/llm_datasets/LLaVA-Pretrain/blip_laion_cc_sbu_558k.json'),
        "image": Path('/workspace/llm_datasets/LLaVA-Pretrain'),
        "flag": "laion",
    }
    
]

target = Path('/workspace/llm_datasets/LLaVA-Pretrain/pretrain_data.json')

output = []
for source in sources:
    with source['json'].open('r', encoding='utf-8') as fp:
        data = json.load(fp)
        for row in tqdm(data):
            if source['flag'] == 'cc3m':
                row['image'] = f"images/{row['image']}"
                
            # image_file = source['image'] / row['image']
            # if not image_file.exists():
            #     print(row)
            #     continue

            output.append(row)


with target.open('w', encoding='utf-8') as fp:
    json.dump(output, fp, indent=4, ensure_ascii=False)

100%|██████████| 558128/558128 [00:00<00:00, 2453243.46it/s]


In [8]:
!du -sh /workspace/llm_datasets/LLaVA-Pretrain/pretrain_data.json

447M	/workspace/llm_datasets/LLaVA-Pretrain/pretrain_data.json


In [2]:
import json
help(json.dump)

Help on function dump in module json:

dump(obj, fp, *, skipkeys=False, ensure_ascii=True, check_circular=True, allow_nan=True, cls=None, indent=None, separators=None, default=None, sort_keys=False, **kw)
    Serialize ``obj`` as a JSON formatted stream to ``fp`` (a
    ``.write()``-supporting file-like object).
    
    If ``skipkeys`` is true then ``dict`` keys that are not basic types
    (``str``, ``int``, ``float``, ``bool``, ``None``) will be skipped
    instead of raising a ``TypeError``.
    
    If ``ensure_ascii`` is false, then the strings written to ``fp`` can
    contain non-ASCII characters if they appear in strings contained in
    ``obj``. Otherwise, all such characters are escaped in JSON strings.
    
    If ``check_circular`` is false, then the circular reference check
    for container types will be skipped and a circular reference will
    result in an ``RecursionError`` (or worse).
    
    If ``allow_nan`` is false, then it will be a ``ValueError`` to
    seriali